## FORMAL METHODS FOR CYBER-PHYSICAL SYSTEMS
### Final Project
#### Umberto Andria Mtr. 1166612

The target of the project was to implement an algorithm to perform the model checking of a specific class of LTL formula: the reactivity class. The reactivity class has the following form:

$\Box\Diamond f \to \Box\Diamond g$

Where f and g are boolean combination of base formula without temporal operators.

In order to check if all the executions of the loaded model satisfy or not the LTL formula, the first step is to negate the formula. The negation of a reactivity formula is:

$\Box\Diamond f \to \Diamond\Box \neg g$

The second step is to check if there is an execution in the model that satisfies the negated formula.

The function that performs this kind of check is the following:

In [4]:
def my_check_react_spec(spec):
    "Pre-defined function to parse a reactivity formula"
    spec_parsed = parse_react(spec)
    "Check if `spec` is a reactivity formula"
    if spec_parsed == None:
        "return None if not. The algorithm stops here"
        return None
    "Otherwise get the model as bddFsm"
    fsm = get_model_encoded_with_bdd()
    "convert the specifications f and g into bdd data structure"
    spec_f = spec_to_bdd(fsm, spec_parsed[0])
    spec_g = spec_to_bdd(fsm, spec_parsed[1])
    "and call the model checking function"
    execution = reactivity_spec(fsm, spec_f, spec_g)
    "If the first element of the tuple `execution` is False"
    if execution[0] == False:
        "build the counterexample"
        witness = build_witness(fsm, execution[1])
        return False, witness
    return True, None

The core of this procedure is the call to the function `reactivity_spec(fsm, spec_f, spec_g)`. This auxiliary function performs a symbolic nested search algorithm for checking the repeatability of `spec_f` and the persistence of `spec_g` negated. The code is the following:


In [ ]:
def reactivity_spec(fsm, spec_f, spec_g):
    reach = fsm.init
    new = reach
    trans = fsm.trans
    "Loop to perform the reachability states calculation"
    while fsm.count_states(new) != 0:
        post_region = trans.post(new)
        new = post_region - reach
        reach = reach | new
    "Region of all reacheable states that satisfy `spec_f`"
    recur = reach & spec_f
    "Region that represent the transition relations"
    trans = fsm.trans
    "The point is to find a cycling execution that satisfy `spec_f` and not satisfy `spec_g`"
    while fsm.count_states(recur) != 0:
        reach = pynusmv.dd.BDD.true() - pynusmv.dd.BDD.true()
        new = trans.pre(recur) & (~spec_g) 
        while fsm.count_states(new) != 0:
            reach = reach | new
            if recur.entailed(reach):
                witness = build_witness(fsm, reach)
                return False, reach
            pre = trans.pre(new) & (~spec_g)
            new = pre - reach
        recur = recur & reach
    return True, None

With the first loop, the function calculates the region that contains those states that are reacheable from the initial states. With the last two nested loops the function tries to find a cycling execution that satisfy the negated LTL formula. The important bit is that in the pre-image calculation the region is intersected with the negation of the `spec_g` formula. In this way the algorithm is sure that the states in the pre-image region always satisfy the negated `spec_g`. So, if a cycling execution is found, we can be sure that: not only `spec_f` is always eventually satisfied but also that not `spec_g` is eventually always satisfied.

The function returns False and the region `reach`, which contains the cycling execution, or True and None if the execution is not found, meaning that the model satisfy in every execution the LTL formula.

If a cycling execution is found `my_check_react_spec(spec)` calls another function `build_witness(fsm, reach)`. This one create the execution as a tuple of alternating states and inputs using the `reach` region.


In [ ]:
def build_witness(fsm, reach):
    states = []
    while fsm.count_states(reach):
        state = fsm.pick_one_state(reach)
        states.append(state)
        reach = reach - state
    last_state = states[len(states) -1]
    states.reverse()
    states.append(last_state)
    witness = ()
    for i in range(0, len(states) -1):
        inputs = fsm.get_inputs_between_states(states[i], states[i+1])
        if inputs != pynusmv.dd.BDD.false():
            input = fsm.pick_one_input(inputs)
            witness += (states[i].get_str_values(),) + (input.get_str_values(),)
        else:
            witness += (states[i].get_str_values(),) + ({},)
    witness += (states[len(states) -1].get_str_values(),)
    return witness

The key point here is that we reverse the list of states because when searching for the cycling execution we have used the pre-image calculation, exploring the states backwards respect the normal execution flow.